In [ ]:
import os.path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from dbispipeline.analytics import get_results_as_dataframe

results = get_results_as_dataframe(project_name='hit-prediction-ismir2020', table_name='hit_prediction')
results = results[results['date'] > '2020-08-01']
results['mean'] = results['outcome'].apply(lambda v: pd.DataFrame(v['mean']))
results['approach'] = results['sourcefile'].apply(lambda v: os.path.splitext(os.path.basename(v))[0].replace('unique_', '').replace('wide_and_deep', 'wd'))

best_results = []
for _, row in results.iterrows():
    row = pd.DataFrame(pd.concat([row, row['mean'].max(axis=1).abs()], axis=0)).transpose()

    best_results.append(row)
best_results = pd.concat(best_results)

best_results['mae'] = best_results['neg_mean_absolute_error']
best_results['rmse'] = best_results['neg_mean_squared_error'].pow(1./2)
best_results = best_results.sort_values(by=['approach'])

display(best_results[['date', 'approach', 'mae', 'rmse', 'hit_nonhit_accuracy']])

display(best_results[['approach', 'mae']].plot.bar(x='approach', y='mae', title='Wide and Deep MAE', figsize=(12, 6)))
display(best_results[['approach', 'rmse']].plot.bar(x='approach', y='rmse', title='Wide and Deep RMSE', figsize=(12, 6)))
display(best_results[['approach', 'hit_nonhit_accuracy']].plot.bar(x='approach', y='hit_nonhit_accuracy', title='Wide and Deep Hit/Non-Hit Accuracy', figsize=(12, 6)))